# 1. Grabcut

- 초기에 전경(foreground)을 포함하는 대략적인 사각형 영역을 지정  
  - 사각형 외부는 확실한 배경(sure background)으로 판단
  - 사각형 내부는 알 수 없는(unknown) 상태
- 보다 디테일한 분할은 사용자가 컴퓨터에게 알려줌  
  - 0: 확실한 배경
  - 1: 확실한 전경
- mask는 내부적으로 4개의 flag값을 가짐
  - cv2.GC_BGD (0): 확실한 배경
  - cv2.GC_FGD (1): 확실한 전경
  - cv2.GC_PR_BGD (2): 배경일 가능성
  - cv2.GC_PR_FGD (3): 전경일 가능성


### [reference]
[link #1 - application code](https://m.blog.naver.com/PostView.nhn?blogId=samsjang&logNo=220606250662&proxyReferer=https%3A%2F%2Fwww.google.com%2F)   
[link #2 - opencv](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_grabcut/py_grabcut.html)

# 2. Pactice

In [1]:
import numpy as np
import cv2

BLUE, GREEN, RED, BLACK, WHITE = (255,0,0), (0,255,0), (0,0,255), (0,0,0), (255,255,255)
DRAW_BG = {'color': BLACK, 'val': 0}
DRAW_FG = {'color': WHITE, 'val': 1}

rect = (0,0,1,1)
drawing = False
rectangle = False
rect_over = False
rect_or_mask = 100
value = DRAW_FG
thickness = 3

def onMouse(event, x, y, flags, param):
    global ix, iy, img, img2, drawing, value, mask, rectangle
    global rect, rect_or_mask, rect_over
    
    if event == cv2.EVENT_RBUTTONDOWN:
        rectangle = True
        ix, iy = x, y
    
    elif event == cv2.EVENT_MOUSEMOVE:
        if rectangle:
            img = img2.copy()
            cv2.rectangle(img, (ix,iy), (x,y), RED, 2)
            rect = (min(ix,x), min(iy,y), abs(ix-x), abs(iy-y))
            rect_or_mask = 0
    
    elif event == cv2.EVENT_RBUTTONUP:
        rectangle = False
        rect_over = True
        
        cv2.rectangle(img, (ix,iy,), (x,y), RED, 2)
        rect = (min(ix,x), min(iy,y), abs(ix-x), abs(iy-y))
        rect_or_mask = 0
        
    
    if event == cv2.EVENT_LBUTTONDOWN:
        if not rect_over:
            print('Draw a rectangle first using the right mouse button')
        
        else:
            drawing = True
            cv2.circle(img, (x,y), thickness, value['color'], -1)
            cv2.circle(mask, (x,y), thickness, value['val'], -1)
        
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            cv2.circle(img, (x,y), thickness, value['color'], -1)
            cv2.circle(mask, (x,y), thickness, value['val'], -1)
            
    elif event == cv2.EVENT_LBUTTONUP:
        if drawing:
            drawing = False
            cv2.circle(img, (x,y), thickness, value['color'], -1)
            cv2.circle(mask, (x,y), thickness, value['val'], -1)
        
    

def grabcut():
    global ix, iy, img, img2, drawing, value, mask, rectangle
    global rect, rect_or_mask, rect_over
    
    img = cv2.imread('foot.jpg')
    img = cv2.resize(img, None, fx=0.5, fy=0.5)
    img2 = img.copy()
    
    mask = np.zeros(img.shape[:2], dtype=np.uint8)
    output = np.zeros(img.shape, np.uint8)
    
    cv2.namedWindow('input')
    cv2.namedWindow('output')
    cv2.setMouseCallback('input', onMouse, param=(img, img2))

    while True:
        cv2.imshow('input', img)
        cv2.imshow('output', output)
        
        k = cv2.waitKey(1) & 0xff
        
        if k == 27:
            break
            
        if k == ord('0'):
            print('>> for foreground')
            value = DRAW_BG
        elif k == ord('1'):
            print('>> for background')
            value = DRAW_FG
        elif k == ord('r'):
            print('>> reset')
            rect = (0,0,1,1)
            drawing = False
            rectangle = False
            rect_or_mask = 100
            rect_over = False
            value = DRAW_FG
            img = img2.copy()
            mask = np.zeros(img.shape[:2], dtype=np.uint8)
            output = np.zeros_like(img, dtype=np.uint8)
        elif k == ord('n'):
            bgdModel = np.zeros((1,65), np.float64)
            fgdModel = np.zeros((1,65), np.float64)
            
            if rect_or_mask == 0:
                cv2.grabCut(img2, mask, rect, bgdModel, fgdModel, 1, cv2.GC_INIT_WITH_RECT)
                rect_or_mask = 1
            elif rect_or_mask == 1:
                cv2.grabCut(img2, mask, rect, bgdModel, fgdModel, 1, cv2.GC_INIT_WITH_MASK)
                
        mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
        output = img2*mask2[:,:,np.newaxis]

#         ## mask 확인용
#         tmp = np.full_like(mask, 128)
#         tmp[mask==0] = 0
#         tmp[mask==1] = 255
#         cv2.imshow('mask', tmp)

#     print(np.unique(mask))
    
    cv2.destroyAllWindows()

In [2]:
grabcut()